
# Arguments

In [1]:
#load from last improved and do 1 epoch only


batch_size = 32
dataset_path_plantvillage = '../datasets/plantvillage'
dataset_path_self = '../datasets/self'
dataset_path_validation = '../datasets/validation'

dataset_paths = [dataset_path_plantvillage, dataset_path_self, dataset_path_validation]

model_save_path = 'model-final.h5'

checkpoint_path = 'checkpoints-final.hdf5'

input_width = 224
input_height = 224
input_depth = 3

# --------------------------------------------------
num_of_epochs = 1
start_epoch = 0
# --------------------------------------------------

# Print class names

In [2]:
# Get classes
import os
import re

class_names = []
class_paths_plant_village = []
class_paths_plant_self = []
class_paths_plant_validation = []

for path in dataset_paths:
    classes = os.listdir(path)
    for i in classes:
        if(re.search('___', i)):
            if(i not in class_names):
                class_names.append(i)
            if(re.search('plantvillage', path)):
                class_paths_plant_village.append(path + '/' + i)
            elif(re.search('self', path)):
                class_paths_plant_self.append(path + '/' + i)
            else:
                class_paths_plant_validation.append(path + '/' + i)

print(class_names)   
print()
print('PlantVillage')
print(class_paths_plant_village) 
print(len(class_paths_plant_village))
print()
print('Self')
print(class_paths_plant_self) 
print(len(class_paths_plant_self))
print()
print('Validation')
print(class_paths_plant_validation) 
print(len(class_paths_plant_validation))

['___Yellow_Leaf_Curl_Virus', '___Late_blight', '___Early_blight', '___Two-spotted_spider_mite', '___Septoria_leaf_spot', '___Bacterial_spot', '___healthy', '___Target_Spot', '___Mosaic_Virus', '___Leaf_Mold', '___Leaf_miner', '___Curly_top_virus', '___Appids']

PlantVillage
['../datasets/plantvillage/___Yellow_Leaf_Curl_Virus', '../datasets/plantvillage/___Late_blight', '../datasets/plantvillage/___Early_blight', '../datasets/plantvillage/___Two-spotted_spider_mite', '../datasets/plantvillage/___Septoria_leaf_spot', '../datasets/plantvillage/___Bacterial_spot', '../datasets/plantvillage/___healthy', '../datasets/plantvillage/___Target_Spot', '../datasets/plantvillage/___Mosaic_Virus', '../datasets/plantvillage/___Leaf_Mold']
10

Self
['../datasets/self/___Leaf_miner', '../datasets/self/___Early_blight', '../datasets/self/___Curly_top_virus', '../datasets/self/___Appids']
4

Validation
['../datasets/validation/___Leaf_miner', '../datasets/validation/___Yellow_Leaf_Curl_Virus', '../data

# Select training classes

early blight in plantVillage dataset replaced by self one

In [3]:
class_paths_training = ['../datasets/plantvillage/___Yellow_Leaf_Curl_Virus', '../datasets/plantvillage/___Late_blight', '../datasets/plantvillage/___Early_blight',  '../datasets/plantvillage/___Bacterial_spot', '../datasets/plantvillage/___healthy', '../datasets/plantvillage/___Target_Spot', '../datasets/plantvillage/___Mosaic_Virus', '../datasets/plantvillage/___Leaf_Mold']
class_names = None

# Load dataset

In [4]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

def load_datasets(path_list):
    # Load image paths
    image_paths = []
    print("[INFO] loading images...")
    for path in path_list:
        image_paths.extend(list(paths.list_images(path)))
        
    # Get unique classnames

    class_names = [pt.split(os.path.sep)[-2] for pt in image_paths]
    class_names = [str(x) for x in np.unique(class_names)]

    # Initial image preprocessing
    aap = AspectAwarePreprocessor(input_width, input_height)
    iap= ImageToArrayPreprocessor()

    #Load image data and perform image data preprocessing
    sdl = SimpleDatasetLoader(preprocessors=[aap,iap])
    (data,labels)  = sdl.load(image_paths,verbose=500)
    data = data.astype("float") / 255.0


    # train test split
    (train_x,test_x,train_y,test_y) = train_test_split(data,labels,test_size=0.25,random_state=42)

    # convert the labels from integers to vectors
    train_y = LabelBinarizer().fit_transform(train_y)
    test_y = LabelBinarizer().fit_transform(test_y)
    
    return (train_x,test_x,train_y,test_y, class_names)

(train_x,test_x,train_y,test_y, class_names) = load_datasets(class_paths_training)



Using TensorFlow backend.


[INFO] loading images...
[INFO]: Processed 500/14713
[INFO]: Processed 1000/14713
[INFO]: Processed 1500/14713
[INFO]: Processed 2000/14713
[INFO]: Processed 2500/14713
[INFO]: Processed 3000/14713
[INFO]: Processed 3500/14713
[INFO]: Processed 4000/14713
[INFO]: Processed 4500/14713
[INFO]: Processed 5000/14713
[INFO]: Processed 5500/14713
[INFO]: Processed 6000/14713
[INFO]: Processed 6500/14713
[INFO]: Processed 7000/14713
[INFO]: Processed 7500/14713
[INFO]: Processed 8000/14713
[INFO]: Processed 8500/14713
[INFO]: Processed 9000/14713
[INFO]: Processed 9500/14713
[INFO]: Processed 10000/14713
[INFO]: Processed 10500/14713
[INFO]: Processed 11000/14713
[INFO]: Processed 11500/14713
[INFO]: Processed 12000/14713
[INFO]: Processed 12500/14713
[INFO]: Processed 13000/14713
[INFO]: Processed 13500/14713
[INFO]: Processed 14000/14713
[INFO]: Processed 14500/14713


In [5]:
print('Dataset loaded')
print(class_names)

Dataset loaded
['___Bacterial_spot', '___Early_blight', '___Late_blight', '___Leaf_Mold', '___Mosaic_Virus', '___Target_Spot', '___Yellow_Leaf_Curl_Virus', '___healthy']



# Load model

In [6]:
from keras.applications import VGG16
from keras.utils.vis_utils import plot_model
from keras.layers import Input

base_model = VGG16(weights=None,include_top=False,
                  input_tensor=Input(shape = (input_width,input_height, input_depth)))

# plot_save_path = 'diagram-base-resnet50.png'
# plot_model(base_model, to_file=plot_save_path, show_shapes=True)

# Add head section

In [7]:
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense

head_model = base_model.output
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(1024,activation='relu')(head_model)
# Add a softmaxc layer
head_model = Dense(len(class_names),activation='softmax')(head_model)

In [8]:
from keras.models import Model


# place the head FC model on top of the base model 
model = Model(inputs=base_model.input, outputs = head_model)

plot_save_path = 'diagram-vgg16.png'
# plot_model(model, to_file=plot_save_path, show_shapes=True)

# Compile model

In [9]:
from keras.optimizers import SGD

# Build the model from the new
print("[INFO] re-compiling model ...")
opt = SGD(lr=0.001, momentum=0.09)
# Fine-tuning with a small learning rate
model.compile(loss = 'categorical_crossentropy',optimizer = opt,
              metrics=['accuracy'])


[INFO] re-compiling model ...


In [10]:
print(model.metrics_names)

['loss', 'accuracy']


# Checkpoints

In [11]:
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', 
save_best_only=True, verbose=1)

callbacks = [checkpoint]



In [12]:
#load checkpoints if existing

import os

if(os.path.exists(checkpoint_path)):
    model.load_weights(checkpoint_path)

# Training monitor

In [13]:
from utils.callbacks.training_monitor import TrainingMonitor
import pathlib
import json
import os

fig_path = "plot"
json_path = "values.json"
values_path = 'values.json'

callbacks.append(TrainingMonitor(fig_path, json_path, start_epoch))

# Warm up head

In [14]:
model.fit_generator(
    aug.flow(train_x,train_y, batch_size = batch_size),
             validation_data = (test_x,test_y),
             epochs=num_of_epochs,
             steps_per_epoch = len(train_x) //32,
             verbose = 1,
             callbacks=callbacks)

Epoch 1/1
344/344 [==============================] - 337s 980ms/step - loss: 0.1168 - accuracy: 0.9609 - val_loss: 0.1555 - val_accuracy: 0.9519

Epoch 00001: val_loss improved from inf to 0.15547, saving model to checkpoints-final.hdf5


In [15]:
model.save(model_save_path)

# Evaluate with 20% test set from same dataset

In [16]:
from sklearn.metrics import classification_report

print("[INFO] evaluating with test set...")
predictions = model.predict(test_x,batch_size=batch_size)
print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),target_names=class_names))

[INFO] evaluating with test set...
                           precision    recall  f1-score   support

        ___Bacterial_spot       0.96      0.94      0.95       549
          ___Early_blight       0.88      0.88      0.88       263
           ___Late_blight       0.96      0.89      0.92       462
             ___Leaf_Mold       0.96      0.95      0.96       240
          ___Mosaic_Virus       1.00      0.88      0.93        90
           ___Target_Spot       0.84      0.93      0.88       329
___Yellow_Leaf_Curl_Virus       0.99      0.99      0.99      1333
               ___healthy       0.94      1.00      0.97       413

              avg / total       0.95      0.95      0.95      3679



# Evaluate with validation images

In [17]:
class_paths_validation = ['../datasets/validation/___Yellow_Leaf_Curl_Virus', '../datasets/validation/___Late_blight', '../datasets/validation/___Early_blight',  '../datasets/validation/___Bacterial_spot', '../datasets/validation/___healthy', '../datasets/validation/___Target_Spot', '../datasets/validation/___Mosaic_Virus', '../datasets/validation/___Leaf_Mold']

In [18]:
(train_x,test_x,train_y,test_y, class_names) = load_datasets(class_paths_validation)

[INFO] loading images...


In [19]:

from sklearn.metrics import classification_report

print("[INFO] evaluating with validation set...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))

[INFO] evaluating with validation set...
                           precision    recall  f1-score   support

        ___Bacterial_spot     0.0000    0.0000    0.0000        10
          ___Early_blight     0.1471    0.3333    0.2041        15
           ___Late_blight     0.4167    0.7353    0.5319        34
             ___Leaf_Mold     0.0000    0.0000    0.0000        29
          ___Mosaic_Virus     0.0000    0.0000    0.0000         1
           ___Target_Spot     0.0000    0.0000    0.0000         6
___Yellow_Leaf_Curl_Virus     0.0000    0.0000    0.0000         7
               ___healthy     0.0000    0.0000    0.0000         5

              avg / total     0.1530    0.2804    0.1976       107



/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
